Importation des librairies

In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
import os
from langchain.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [15]:
pdf_loader = PyPDFLoader("data/code_penal.pdf")
pages = pdf_loader.load()

In [16]:
pages[0]

Document(page_content="Code pénal\nPartie législative\nLivre Ier : Dispositions générales\nTitre Ier : De la loi pénale\nChapitre Ier : Des principes généraux\nArticle 111-1\n \nLes infractions pénales sont classées, suivant leur gravité, en crimes, délits et contraventions.\n \n \nArticle 111-2\n \nLa loi détermine les crimes et délits et fixe les peines applicables à leurs auteurs.\n \n \nLe règlement détermine les contraventions et fixe, dans les limites et selon les distinctions établies par la loi,\nles peines applicables aux contrevenants.\n \nArticle 111-3\n \nNul ne peut être puni pour un crime ou pour un délit dont les éléments ne sont pas définis par la loi, ou pour\nune contravention dont les éléments ne sont pas définis par le règlement.\n \n \nNul ne peut être puni d'une peine qui n'est pas prévue par la loi, si l'infraction est un crime ou un délit, ou par\nle règlement, si l'infraction est une contravention.\n \nArticle 111-4\n \nLa loi pénale est d'interprétation strict

In [17]:
texts = [doc.page_content for doc in pages]

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

split_text = text_splitter.create_documents(texts)
print(split_text[0])

page_content="Code pénal\nPartie législative\nLivre Ier : Dispositions générales\nTitre Ier : De la loi pénale\nChapitre Ier : Des principes généraux\nArticle 111-1\n \nLes infractions pénales sont classées, suivant leur gravité, en crimes, délits et contraventions.\n \n \nArticle 111-2\n \nLa loi détermine les crimes et délits et fixe les peines applicables à leurs auteurs.\n \n \nLe règlement détermine les contraventions et fixe, dans les limites et selon les distinctions établies par la loi,\nles peines applicables aux contrevenants.\n \nArticle 111-3\n \nNul ne peut être puni pour un crime ou pour un délit dont les éléments ne sont pas définis par la loi, ou pour\nune contravention dont les éléments ne sont pas définis par le règlement.\n \n \nNul ne peut être puni d'une peine qui n'est pas prévue par la loi, si l'infraction est un crime ou un délit, ou par\nle règlement, si l'infraction est une contravention.\n \nArticle 111-4\n \nLa loi pénale est d'interprétation stricte.\n \n \

In [19]:
def format_docs(docs):
    # Vérifier que docs est une liste de documents
    assert isinstance(docs, list), f"Expected list, got {type(docs)}"
    return "\n\n".join([doc.page_content for doc in docs])

In [20]:
load_dotenv()

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))

vstore = Chroma.from_documents(split_text,embeddings)

# Définir le modèle LLM d'OpenAI
llm = ChatOpenAI(model_name="gpt-4", verbose=True, openai_api_key=os.getenv('OPENAI_API_KEY'))

# Créer le récupérateur de documents
retriever = vstore.as_retriever()

In [21]:
# Définir le modèle de sortie Pydantic
class QAOutput(BaseModel):
    answer: str = Field(..., description="The summary answer to the question")
    sources: str = Field(..., description="The sources used to generate the answer")

# Créer le parser Pydantic
output_parser = PydanticOutputParser(pydantic_object=QAOutput)


In [22]:
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Voici des extraits du code pénal : {context}\n\n"
        "Question : {question}\n\n"
        "Répondez à la question en fournissant un résumé synthétique. "
        "Si tu ne trouves pas de réponse, ne répond pas et répond que l'information n'est pas contenue dans le code pénal. "
        "Liste les articles utilisés pour générer la réponse."
    )
)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=lambda x: {"context": format_docs(x["context"]), "question": x["question"]})
    | prompt_template
    | llm
    | StrOutputParser()
)

In [23]:
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [30]:
question = "Quels est la peine de prison encourue pour le trafic de drogue ?"
response = rag_chain_with_source.invoke(question)

# Afficher la réponse et les sources
print("Answer:", response['answer'])
print("Sources:", response['context'])

Answer: La peine de prison encourue pour le trafic de drogue, selon l'article 222-35 du code pénal, est de vingt ans de réclusion criminelle. Cependant, si les faits sont commis en bande organisée, la peine est de trente ans de réclusion criminelle.
Sources: [Document(page_content="Article 222-35\n \nLa production ou la fabrication illicites de stupéfiants sont punies de vingt ans de réclusion criminelle et de 7\n500 000 euros d'amende.\n \n \nCes faits sont punis de trente ans de réclusion criminelle et de 7 500 000 euros d'amende lorsqu'ils sont\ncommis en bande organisée.\n \n \nCode pénal - Dernière modification le 23 mai 2024 - Document généré le 30 mai 2024"), Document(page_content="Article 222-35\n \nLa production ou la fabrication illicites de stupéfiants sont punies de vingt ans de réclusion criminelle et de 7\n500 000 euros d'amende.\n \n \nCes faits sont punis de trente ans de réclusion criminelle et de 7 500 000 euros d'amende lorsqu'ils sont\ncommis en bande organisée.\n \